In [114]:
import pandas as pd
import nltk.tokenize 
import os
import nltk.tokenize.toktok

In [116]:
path = '../vacancies'
df = pd.concat((pd.read_csv(path + '/' + f) for f in os.listdir(path)), ignore_index=True)

In [118]:
# Choose the type of the work that you want to see according to the following mapping:
#    "{'id': 'probation', 'name': 'Стажировка'}",
#    "{'id': 'full', 'name': 'Полная занятость'}",
#    "{'id': 'part', 'name': 'Частичная занятость'}",
#    "{'id': 'project', 'name': 'Проектная работа'}"], 
for arr in df['employment']: 
    if (eval(arr)['id']) != 'probation':
        df.drop(df[df['employment'] == arr].index, inplace=True)

 
# Optimize this for your needs, i.e. provide some words that you 
# don't want to see in the name of the vacancy
for name in df['name']:
    for word in nltk.tokenize.wordpunct_tokenize(name):
        if word.lower() in ['java', 'php', 'qa', 'backend', 'back-end', 'frontend', 'front-end', 'full-stack',
                            'fullstack' ,'c#']:
            df.drop(df[df['name'] == name].index, inplace=True)

In [119]:
df.dropna(axis=1, how='all', inplace=True)
df.drop(columns=['Unnamed: 0', 'id', 'premium', 'department', 'archived', 'is_adv_vacancy', 'working_days', 
                 'working_time_intervals', 'working_time_modes', 'relations', 'apply_alternate_url', 
                 'accept_incomplete_resumes', 'response_letter_required', 'type', 'area', 
                 'published_at', 'url', 'show_logo_in_search'], inplace=True)
df.fillna('null', inplace=True)

In [120]:
for column in df.columns:
    if column == 'created_at':
        df[column] = pd.to_datetime(df[column])
        df[column] = df[column].dt.strftime('%Y-%m-%d %H:%M')
    if column in ['employer', 'experience', 'employment']:
        df[column] = df[column].apply(lambda x: eval(x)['name'] if x != 'null' else x)
    if column == 'salary':
        df = df.assign(salary_from=df.salary.apply(lambda x: eval(x)['from'] if x != 'null' else x))
        df = df.assign(salary_to=df.salary.apply(lambda x: eval(x)['to'] if x != 'null' else x))
        df.drop(columns=['salary'], inplace=True)
    if column == 'address':
        df[column] = df[column].apply(lambda x: eval(x)['city'] if x != 'null' else x)
    if column == 'professional_roles':
        df[column] = df[column].apply(lambda x: eval(x[1:-1])['name'] if x != 'null' else x)
    if column == 'snippet':
        df = df.assign(requirement=df.snippet.apply(lambda x: eval(x)['requirement'] if x != 'null' else x))
        df = df.assign(responsibility=df.snippet.apply(lambda x: eval(x)['responsibility'] if x != 'null' else x))
        df.drop(columns=['snippet'], inplace=True)


In [121]:
df.to_csv('../transformed.csv', index=False)

In [122]:
df

,name,has_test,address,created_at,alternate_url,employer,accept_temporary,professional_roles,experience,employment,salary_from,salary_to,requirement,responsibility
71,Стажер-разработчик программного обеспечения,False,Красноярск,2023-09-21 08:21,https://hh.ru/vacancy/85699126,Сибирские интеграционные системы,False,"Программист, разработчик",Нет опыта,Стажировка,41000,None,"Знание следующих фреймворков: Angular, Spring ...",Изучение технологии и инструментов разработки ...
98,DevOps инженер (стажер),False,Казань,2023-09-06 14:55,https://hh.ru/vacancy/86337688,ГК Innostage,False,DevOps-инженер,Нет опыта,Стажировка,null,null,Наличие фундаментальных знаний ОС и навыки сис...,"Innostage (Инностейдж) – это компания, оказыва..."
105,Cтажер-разработчик Python/C# (Data Engineer),True,Москва,2023-09-13 11:43,https://hh.ru/vacancy/86551216,Mindbox,False,"Программист, разработчик",Нет опыта,Стажировка,35000,35000,"Базовое знание Python, а также статически типи...","Net6+ везде, многое в k8s. Монолит — не легаси..."
108,Стажёр картограф-разметчик (Data Engineering /...,False,Москва,2023-09-27 10:20,https://hh.ru/vacancy/87396117,SberAutoTech,False,Другое,Нет опыта,Стажировка,null,null,3-6 курс МИИГАиК / ГУЗ / географического факул...,"Классы задач, которые необходимо будет выполня..."
113,Junior Python Developer / Data Engineer,False,Санкт-Петербург,2023-09-08 11:00,https://hh.ru/vacancy/86440903,Smart Design,False,"Программист, разработчик",Нет опыта,Стажировка,null,null,"Работа с SQL/NoSQL базами данных: Mongo, Postg...",Принять участие в построении корпоративного <h...
123,Стажер-инженер в МТС FIX (Big Data),False,null,2023-09-29 13:08,https://hh.ru/vacancy/86000921,МТС,False,Другое,Нет опыта,Стажировка,null,null,Уверенное знание SQL. Знание Python. Знание те...,Проектировать и разрабатывать решения интеграц...
138,Стажер data engineer,False,Санкт-Петербург,2023-09-18 10:12,https://hh.ru/vacancy/86839998,КРОК,False,"BI-аналитик, аналитик данных",Нет опыта,Стажировка,null,null,Образование высшее техническое или в процессе ...,Разработка механизмов загрузки и обработки дан...
139,Стажер Data Engineer,False,Москва,2023-09-27 15:04,https://hh.ru/vacancy/87420670,СБЕР,False,"Программист, разработчик",Нет опыта,Стажировка,null,null,"Высшее/незаконченное высшее образование, специ...",Сейчас мы в поиске стажера на вакансию Стажер ...
151,Supply Chain Management Intern / Стажер отдела...,False,Москва,2023-09-06 14:12,https://hh.ru/vacancy/86334482,Procter & Gamble,False,Другое,Нет опыта,Стажировка,90000,None,The job requires you to be based in Russia. Go...,Optimizing & automating logistics with a help ...
